In [64]:
import pandas as pd
import numpy as np

In [65]:
data = pd.read_csv('train.csv')

In [66]:
data  = pd.DataFrame(data)

In [67]:
data = data.dropna()

In [68]:
X = data.drop('label',axis=1)

In [69]:
y=data['label']

In [70]:
import tensorflow as tf

In [71]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [72]:
voc_size=5000

In [73]:
messages=X.copy()

In [74]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [75]:
messages.reset_index(inplace=True)

In [76]:
import nltk
import re
from nltk.corpus import stopwords

In [77]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tejas.raval@simform.dom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [79]:
corpus


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [80]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2225, 4311, 1217, 4930, 2026, 916, 4773, 2982, 513, 575],
 [3566, 4689, 3345, 241, 3267, 2837, 996],
 [1069, 4886, 488, 2530],
 [3638, 1183, 3355, 2156, 1414, 1498],
 [2038, 3267, 2808, 1847, 518, 47, 3267, 3714, 2630, 4484],
 [3654,
  4243,
  2992,
  2070,
  1220,
  417,
  1724,
  4148,
  247,
  1507,
  3709,
  2259,
  2155,
  2753,
  996],
 [2468, 768, 4681, 170, 1304, 3493, 1394, 652, 2547, 725, 3614],
 [1827, 4519, 1619, 2328, 2132, 3229, 417, 1060, 2547, 725, 3614],
 [1814, 2141, 4385, 489, 4643, 2182, 3756, 1542, 417, 3717],
 [3969, 3448, 3920, 747, 2418, 3136, 188, 1551],
 [673, 1822, 2272, 1561, 4896, 1408, 2152, 2143, 710, 2192, 4886],
 [2156, 3415, 2026, 2182, 417, 2132],
 [2315, 1376, 2469, 351, 3875, 3268, 1790, 3993, 2063],
 [3259, 4069, 3014, 1477, 1348, 3282, 2617, 2547, 725, 3614],
 [534, 4971, 4388, 2985, 562, 2547, 725, 3614],
 [679, 1170, 1405, 1371, 3823, 3652, 300, 676, 739, 3643],
 [2263, 2048, 4689],
 [3810, 4732, 2550, 2740, 417, 2754, 4258, 996],
 [1484, 1984

Embedding Representation

In [81]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2982  513  575]
 [   0    0    0 ... 3267 2837  996]
 [   0    0    0 ... 4886  488 2530]
 ...
 [   0    0    0 ... 2547  725 3614]
 [   0    0    0 ... 2537 2209 1574]
 [   0    0    0 ...  155 2586 3131]]


In [82]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2225,
       4311, 1217, 4930, 2026,  916, 4773, 2982,  513,  575], dtype=int32)

LSTM Model


In [83]:
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(LSTM(100))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_4 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [84]:
len(embedded_docs),y.shape

(18285, (18285,))

In [107]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [85]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [86]:

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=26)

Model Training

In [88]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 6s 24ms/step - loss: 0.3453 - accuracy: 0.8327 - val_loss: 0.1765 - val_accuracy: 0.9220
Epoch 2/10
200/200 [==============================] - 4s 21ms/step - loss: 0.1519 - accuracy: 0.9413 - val_loss: 0.1737 - val_accuracy: 0.9254
Epoch 3/10
200/200 [==============================] - 4s 21ms/step - loss: 0.1144 - accuracy: 0.9581 - val_loss: 0.1914 - val_accuracy: 0.9256
Epoch 4/10
200/200 [==============================] - 4s 22ms/step - loss: 0.0937 - accuracy: 0.9665 - val_loss: 0.1995 - val_accuracy: 0.9247
Epoch 5/10
200/200 [==============================] - 4s 21ms/step - loss: 0.0729 - accuracy: 0.9760 - val_loss: 0.2364 - val_accuracy: 0.9249
Epoch 6/10
200/200 [==============================] - 4s 22ms/step - loss: 0.0616 - accuracy: 0.9789 - val_loss: 0.2861 - val_accuracy: 0.9264
Epoch 7/10
200/200 [==============================] - 4s 21ms/step - loss: 0.0478 - accuracy: 0.9845 - val_loss: 0.2796 - val_accuracy: 0.9222

Adding Dropout

In [89]:
from tensorflow.keras.layers import Dropout # type: ignore
## Creating model


Performance Metrics And Accuracy

In [111]:
predictions = model.predict(X_test)
y_pred = (predictions.flatten() >= 0.5).astype(int)

172/172 [==============================] - 1s 3ms/step


In [112]:
y_test

array([1, 1, 0, ..., 1, 0, 1])

In [113]:
y_pred

array([1, 1, 0, ..., 1, 0, 1])

In [105]:
from sklearn.metrics import confusion_matrix

In [114]:

confusion_matrix(y_test,y_pred)

array([[2881,  228],
       [ 224, 2153]])

In [115]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9176084578928181